# Pitch Extraction Simulation notebook 

This notebook is for testing of simulation blocks

In [2]:
import pyaudio
import struct
import matplotlib.pyplot as plt
import matplotlib.animation as animation
# from matplotlib import style
import numpy as np
import sys, select, os
import time
%matplotlib qt

FORMAT = pyaudio.paInt16
CHANNELS = 1 # Mono audio
SAMPLE_RATE = 44100 # Sample rate
WINDOW_SAMPLES = 2048 # Each window will be 2048 samples long at 22.68us per sample = 0.046s per window
WINDOWS_PER_BUFFER = 1
FRAMES_PER_BUFFER = WINDOW_SAMPLES*WINDOWS_PER_BUFFER

In [3]:
p = pyaudio.PyAudio()
streamIn = p.open(
        format = FORMAT,
        channels = CHANNELS,
        rate = SAMPLE_RATE,
        input = True,
        output = False,
        frames_per_buffer = FRAMES_PER_BUFFER
        )

In [4]:
# Slower method
# %timeit data_int = struct.unpack(str(FRAMES_PER_BUFFER)+'h',data) 

### AMDF Pitch extractor function

In [23]:
# Function is defined as Dn = 1/L * sum^{L}_{k=0} |S_{j} - S{j-n}| with n = 0,1,2,3... 
#                                                              and Sj are the samples
def amdf_PE(inputWindow):
    D_tau = np.zeros((8,256))
    minIndices = np.zeros(8)
    freq = np.zeros(8)
    vol = np.zeros(8)
#     scale_factor = 1/len(inputWindow)
    
    for c in range(8):
        inputWindow_block = inputWindow[c*256:(c+1)*256]
        tau = np.arange(0,(len(inputWindow_block)-1))
        for i,t in enumerate(tau):
            temp = 0
            shifted = np.zeros_like(inputWindow_block)
            if t == 0:
                shifted = inputWindow_block
            else:
                shifted[:t] = 0
                shifted[t:] = inputWindow_block[:-t]
    #         for i,win in enumerate(inputWindow):
    #         temp =+ np.abs(win-shifted[i])
            diffArr = np.abs(np.subtract(inputWindow_block,shifted))
            temp = (np.sum(diffArr)/len(inputWindow_block))
            D_tau[c,i] = temp
        offset = np.argmax(D_tau[c,:])
        minIndices[c] = (c*256+offset)+np.argmin(D_tau[c,offset:-1])
        freq[c] = SAMPLE_RATE/(minIndices[c]-(c*256))
        vol[c] = (np.mean(np.abs(inputWindow_block))/2**15)*100
        
#     minIndices[0] = np.argmin(D_tau[1:1024])
#     minIndices[1] = 1024+np.argmin(D_tau[1025:2048])
#     minIndices[2] = 2048+np.argmin(D_tau[2049:4096])
#     print(minIndices)
#     for i, data in enumerate(D_tau): 
#         try:
#             if (D_tau[i-1]>data) and (D_tau[i+1]> data):
#                 minIndices.append(i)
#         except:
#             if (D_tau[i-1])>data: 
#                 minIndices.append(i)
    return D_tau.flatten(),minIndices, freq, vol

In [38]:
from sklearn.preprocessing import normalize
x = np.random.rand(100)

norm2 = normalize(x[:,np.newaxis], axis = 0).ravel()
plt.plot(x)
plt.plot(norm1)
plt.plot(norm2)

In [23]:
a = np.array([19,2,13,45,5,6,7,28,9,5,5,5,31,11,1,52,5,5])
vals, count = np.unique(a,return_counts = True)
index = count.argmax()
vals[index]

5

### Stretch function
Trying to determine an on-the-fly function generator that will compensate for the decay of the AMDF. The linear function does not work as the decay is not linear and approximation with a linear function does not yield good results yet

Possible fixes:
    - scaling factor in the denominator
    - non-linear function (will require some form of interpolation)

In [6]:
## Stretch function (not currently working)
D_max = np.max(D_tau)
stretch = np.zeros(FRAMES_PER_BUFFER)
stretch[0] = 1
for n in range(1,FRAMES_PER_BUFFER):
    stretch[n] = D_max*((-1/(10*2*FRAMES_PER_BUFFER))*n)
D_tau = (D_tau*stretch-(D_max/2))

NameError: name 'D_tau' is not defined

In [43]:
plt.plot(stretch)

In [7]:
data = streamIn.read(FRAMES_PER_BUFFER)
data_int = np.frombuffer(data,dtype='<i2')
# print(data_int)
a = time.time()
AMDF_data = amdf_PE(data_int)
b = time.time()
print("Time taken: {}".format(b-a))
# plt.plot(AMDF_data)

Time taken: 0.026920795440673828


### Plotting function

In [53]:
fig, ax = plt.subplots()
ax.set_title("AMDF")
ax.set_ylim([-(2**8),(2**12)])
vline0 = ax.axvline(x =   0.,color = 'red')
vline01 = ax.axvline(x =   0.,color = 'red')
vline1 = ax.axvline(x = 256,color = 'k')
vline01 = ax.axvline(x = 256,color = 'k')
vline2 = ax.axvline(x = 512,color = 'green')
vline02 = ax.axvline(x = 512,color = 'green')
vline3 = ax.axvline(x = 768,color = 'orange')
vline03 = ax.axvline(x = 768,color = 'orange')
vline4 = ax.axvline(x = 1024,color = 'pink')
vline04 = ax.axvline(x = 1024,color = 'pink')
vline5 = ax.axvline(x = 1280,color = 'purple')
vline05 = ax.axvline(x = 1280,color = 'purple')
vline6 = ax.axvline(x = 1536,color = 'yellow')
vline06 = ax.axvline(x = 1536,color = 'yellow')
vline7 = ax.axvline(x = 1792,color = 'cyan')
vline07 = ax.axvline(x = 1792,color = 'cyan')
line, = ax.plot(np.zeros(FRAMES_PER_BUFFER))

# line1, = ax.plot(np.zeros(FRAMES_PER_BUFFER))
# line2, = ax.plot(np.zeros(FRAMES_PER_BUFFER))
plt.show()
signal = []
amdf_val = []
stretch = []
freqs = []
vols = []
timerStart = time.time()
timer = 0
while True:
    data = streamIn.read(FRAMES_PER_BUFFER)
    data_int2 = np.frombuffer(data,dtype='<i2')
    amdf_data, vline, freqArr, vol = amdf_PE(data_int2)
    line.set_ydata(amdf_data)
    freqs.append(freqArr)
    vols.append(vol)
    timer = time.time()-timerStart
    if timer>= 3.0:
        finalTimer = time.time()-timerStart
        break
    vline0.set_data(vline[0] , [0,1])
    vline1.set_data(vline[1] , [0,1])
    vline2.set_data(vline[2] , [0,1])
    vline3.set_data(vline[3] , [0,1])
    vline4.set_data(vline[4] , [0,1])
    vline5.set_data(vline[5] , [0,1])
    vline6.set_data(vline[6] , [0,1])
    vline7.set_data(vline[7] , [0,1])
    fig.canvas.draw()
    fig.canvas.update()
    fig.canvas.flush_events()
# plt.close(fig)
freqs = np.array(freqs)
freqs = freqs.flatten()
vols = np.array(vols)
vols = vols.flatten()
vols_norm = np.interp(vols,(vols.min(), vols.max()), (0,100)).astype(int)
print(finalTimer)
xAxis = np.round(np.arange(0,timer,timer/8 ),2)
xticker = np.arange(len(freqs))
fig, ax1 = plt.subplots()
fig.suptitle("Frequency progression of recorded audio")
ax1.set_ylabel("Frequency (Hz)", color = "green")
ax1.set_xlabel("Time (sampled at {} Hz)".format(SAMPLE_RATE))
ax1.plot(freqs, color = 'green')

ax2 = ax1.twinx()
ax2.set_ylabel("Volume as percentage of max (%)", color = "red")
ax2.plot(vols_norm, color = 'red')
plt.xticks(xticker[::66], xAxis)

3.044766902923584


([<matplotlib.axis.XTick at 0x1576742fe10>,
 [Text(0, 0, '0.0'),
  Text(0, 0, '0.38'),
  Text(0, 0, '0.76'),
  Text(0, 0, '1.14'),
  Text(0, 0, '1.52'),
  Text(0, 0, '1.9'),
  Text(0, 0, '2.28'),
  Text(0, 0, '2.66')])

In [58]:
len(xticker[::66]),len(xAxis)

(8, 8)

## Plotting BBBW output for accuracy and speed testing

In [54]:
vols_norm

array([  7,   6,  10,   5,   7,   2,   4,   5,   6,   3,   2,   4,   5,
         1,   1,   3,   2,   0,   2,   5,   0,   0,   0,   1,  65,  46,
        37,  56,  69,  54,  57,  46,  52,  54,  59,  40,  48,  49,  50,
        42,  42,  45,  42,  44,  37,  44,  47,  39,  36,  30,  35,  36,
        35,  31,  30,  25,  34,  27,  25,  22,  28,  22,  25,  35,  27,
        28,  41,  68,  75,  81,  79,  77,  67,  62,  59,  85,  61,  49,
        64,  61,  53,  72,  91,  90,  65,  57,  41,  34,  42,  52,  40,
        32,  49,  41,  50,  51,  61,  49,  65,  41,  61,  45,  46,  40,
        34,  30,  28,  43,  24,  33,  34,  39,  40,  37,  41,  31,  30,
        47,  38,  39,  52,  44,  50,  36,  43,  42,  37,  30,  25,  37,
        32,  48,  35,  29,  30,  29,  27,  30,  41,  24,  24,  21,  23,
        27,  28,  23,  27,  27,  24,  19,  22,  23,  15,  20,  18,  19,
        16,  19,  17,  16,  17,   9,  15,  15,  18,  14,  24,  20,  14,
        16,  15,  16,  14,  27,  35,  29,  36,  31,  30,  31,  3